In [84]:
from path_arquivos import *
import pandas as pd
caminho = output.var_corte
df_corte = pd.read_csv(ar_csv.ar_67,header= None, names= col_name.c67)

In [85]:
def ajuste_col(df_original, col):
    df_copia = df_original.copy() 
    for i in col:
        df_copia[i] = df_copia[i].fillna(0).astype(str)
        df_copia[i] = df_copia[i].str.replace(".", "", regex=False)
        df_copia[i] = df_copia[i].str.replace(",", ".")
        df_copia[i] = df_copia[i].fillna(0).astype(float)
    return df_copia

def correcao(df, dt):
    var = df.copy()

    var['dia'] = var[dt].dt.day_name('pt_BR')
    var['mes'] = var[dt].dt.month_name('pt_BR')
    var['ano'] = var[dt].dt.year
    var[dt] = var[dt].dt.strftime("%d-%m-%Y")     
    var['vl_corte'] = var['vl_corte'].round(2).astype(str).str.replace('.', ',', regex= False)
    var = var.sort_values(by=dt, ascending= True, axis= 0)
    return var

CORPO DO SCRIPTS

In [86]:
df_corte['hora'] = df_corte['hr'].astype(str) + ':' + df_corte['min'].astype(str)
df_corte['data'] = pd.to_datetime(df_corte['data'], format= '%d/%m/%y').sort_index(axis= 0, ascending= True)
df_corte['hora'] = pd.to_datetime(df_corte['hora'], format= '%H:%M').dt.strftime('%H:%M')

col_ajustar = ['vl_corte', 'qtde_corte']
df_corte = ajuste_col(df_corte, col_ajustar)
df_corte = df_corte.sort_values(by="data", ascending= True, axis= 0)
df_dia = df_corte.loc[df_corte['hora'].between("07:30:00", "18:00:00")].copy()

In [ ]:
df_div = df_corte[['data','desc','hora', 'n_ped']].copy()
df_div['data'] = pd.to_datetime(df_div['data'], format= '%d/%m/%Y')
df_div["data_turno"] = df_div['data'].copy()
df_div.loc[df_div['hora'] < "07:30:00", 'data_turno'] -= pd.Timedelta(days=1)
var_div = df_div.groupby('data_turno').agg(
    ped_imperfeito=('n_ped', 'nunique')
).reset_index()
var_div['dia'] = var_div['data_turno'].dt.day_name('pt_BR')
var_div['mes'] = var_div['data_turno'].dt.month_name('pt_BR')
var_div['ano'] = var_div['data_turno'].dt.year
var_div['data_turno'] = var_div['data_turno'].dt.strftime("%d-%m-%Y")


DIVERGENCIA:
    data_turno  ped_imperfeito           dia      mes   ano
0  09-10-2025              23  Quinta-feira  Outubro  2025
1  10-10-2025              70   Sexta-feira  Outubro  2025


In [97]:
# display(df_dia.head())
display(var_div.head())

,data_turno,ped_imperfeito,dia,mes,ano
0,09-10-2025,23,Quinta-feira,Outubro,2025
1,10-10-2025,70,Sexta-feira,Outubro,2025
